# 多线程

In [8]:
from multiprocessing import Pool
import requests
from lxml import etree
import re
import time


headers = {
    'user-agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36"
}


def get_url(link):
    res = requests.get(link, headers=headers)
    selector = etree.HTML(res.text)
    links = selector.xpath('//div[@class="pl2"]/a[1]/@href')
    return links

def get_info(url):
    res = requests.get(url, headers=headers)
    selector = etree.HTML(res.text)
    name = selector.xpath('//*[@id="wrapper"]/h1/span/text()')[0]
    author = re.findall('表演者:.*?>(.*?)</a>', res.text, re.S)[0].strip()
    
    styles = re.findall('<span class="pl">流派:</span>&nbsp;(.*?)<br', res.text, re.S)
    if len(styles) == 0:
        style = '未知'
    else:
        style = styles[0].strip()
        
    medium = re.findall('<span class="pl">介质:</span>&nbsp;(.*?)<br', res.text, re.S)[0].strip()
    publishing_time = re.findall('<span class="pl">发行时间:</span>&nbsp;(.*?)<br', res.text, re.S)[0].strip()
    
    publishers = re.findall('<span class="pl">出版者:</span>&nbsp;(.*?)<br', res.text, re.S)
    if len(publishers) == 0:
        publisher = '未知'
    else:
        publisher = publishers[0].strip()
    
    
    rate = float(selector.xpath('//*[@id="interest_sectl"]/div/div[2]/strong/text()')[0])

    info = { 'name': name,
            'author': author,
            'style': style,
            'medium': medium,
            'publishing_time': publishing_time,
            'publisher': publisher,
            'rate': rate,
    }

    return info


In [5]:
if __name__ == '__main__':
    link = 'https://music.douban.com/top250?start=0'
    urls = get_url(link)
    start_1 = time.time()
    for url in urls:
        get_info(url)
    end_1 = time.time()
    print("串行:", end_1-start_1)
    


串行: 76.35948014259338


NameError: name 'Pool' is not defined

In [9]:
start_2 = time.time()
pool = Pool(processes=2)
pool.map(get_info, urls)
end_2 = time.time()
print("两个进程:", end_2-start_2)

两个进程: 41.03893303871155
